<a href="https://colab.research.google.com/github/akashmehra/AntColonyOptimization/blob/master/LightningLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install PyTorch Lightning

In [ ]:
!pip install pytorch-lightning
!pip install wandb -qqq

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision import transforms
from torch.utils.data import  Dataset
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import os
from collections import Counter
from tqdm.notebook import tqdm
import wandb

In [ ]:
wandb.login(key='') # set key here

In [ ]:
 wandb.init(
      # Set entity to specify your username or team name
      # ex: entity="carey",
      # Set the project where this run will be logged
      entity = "modai",
      project="nlp_tutorials", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 1e-2,
      "architecture": "LSTM",
      "dataset": "WikiText-2",})
  

In [ ]:
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 24 if AVAIL_GPUS else 12
NUM_WORKERS = int(os.cpu_count() / 2)

In [ ]:
wandb_logger = WandbLogger()

### Lightning Model
Here we define the model using PyTorch Lightning's self-contained approach. The model is a very simple **Language Model** which is an `LSTM Encoder` followed by a `Fully Connected Layer` and a `Softmax`. We output the distribution over the tokens to predict the next one given the context.

In [ ]:
class RNNLanguageModel(pl.LightningModule):
    def __init__(self, input_size: int, hidden_size: int = 64, 
                 dropout_prob: float = 0.4, num_layers: int = 2,
                 tie_weights: bool = True):
        super().__init__()
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, 
                           num_layers=num_layers, bias=False, batch_first=True, 
                           bidirectional=False)
        self.dropout = nn.Dropout(dropout_prob)
        self.decoder = nn.Linear(hidden_size, input_size)
        if tie_weights:
            self.decoder.weight = self.encoder.weight

    def forward(self, x):
        encoded = self.encoder(x)
        self.rnn.flatten_parameters()
        output, hidden = self.rnn(encoded)
        return self.decoder(output)
    
    def training_step(self, batch, batch_idx):
        x,y = batch['sources'], batch['targets']
        embedded = self.encoder(x)
        self.rnn.flatten_parameters()
        output, hidden = self.rnn(embedded)
        hx, ctx = hidden
        output = self.dropout(output)
        output = output.reshape(output.size(0) * output.size(1), output.size(2))
        x_hat = self.decoder(output)
        loss = F.cross_entropy(x_hat, y.reshape(-1))
        self.log("training_loss", loss)
        return loss

    def configure_optimizers(self, lr=1e-2):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        return optimizer


In [ ]:
class ToTensor():
    def __call__(self, sample):
        sample['source'] = torch.LongTensor(sample['source'])
        sample['target'] = torch.LongTensor(sample['target'])
        return sample

In [ ]:
class Dictionary(object):
    
    def __init__(self):
        
        self.dont_care = 0
        self.bos_token = 1
        self.eos_token = 2
        self.unk_token = 3
        self.bos_word = '<bos>'
        self.eos_word = '<eos>'
        self.unk_word = '<unk>'
        
        self._word2idx = {self.bos_word: self.bos_token, 
                          self.eos_word: self.eos_token, 
                          self.unk_word: self.unk_token}
        
        self._idx2word = [self.dont_care, self.bos_word, self.eos_word, self.unk_word]
        self._dist = Counter()
        
    def add_word(self, word):
        self._dist[word] += 1
        if word not in self._word2idx:
            self._idx2word.append(word)
            self._word2idx[word] = len(self._idx2word) - 1
        #assert len(self._dist) == len(self._idx2word) == len(self._word2idx)
        return self._word2idx[word]

    def idx_to_word(self, idx):
        return self._idx2word[idx]
    
    def word_to_idx(self, word):
        return self._word2idx[word]
    
    def __add__(self, other):
        pass
        
    def __iadd__(self, other):
        widmap = other.wordidmap
        for k,v in widmap.items():
            if k not in self._word2idx:
                self._word2idx[k] = len(self._word2idx)
                self._idx2word.append(k)
        return self
    
    @property
    def wordidmap(self):
        return self._word2idx
    
    @property
    def idxwordmap(self):
        return self._idx2word
    
    @property
    def word_count(self):
        return self._dist
        

    def __len__(self):
        return len(self._idx2word)


In [ ]:
class SentenceDataset(Dataset):
    def __init__(self, path: str, dictionary: Dictionary, transform = ToTensor()):
        self._dictionary = dictionary
        self._tokens = self._tokenize(path)
        self.transform = transform

    def _tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        doc = []
        tokens = []
        self._num_tokens = 0
        # read lines from file.
        with open(path, 'r', encoding="utf8") as f:
            for line in f:
                if len(line.strip()) == 0:
                    continue
                words = [word.lower() for word in line.strip().split()]
                self._num_tokens += len(words)
                doc.append(words)
                
        for words in tqdm(doc): 
            tokens.append([self._dictionary.word_to_idx(word) for word in words])
                
        return tokens
    
    def __len__(self):
        return len(self._tokens)
    
    @property
    def corpus_size(self):
        return self._num_tokens
    
    def numpy(self):
        samples = []
        transform = ToNumpy()
        for idx in range(len(self._tokens)):
            sample = self[idx]
            sample = transform(sample)
            samples.append(sample)
        return samples
    
    @property
    def vocab(self):
        return self._dictionary
        
    def sentence(self, idx):
        return ' '.join([self._dictionary.idx_to_word(token) for token in self._tokens[idx]])

    def __getitem__(self, idx):
        tokens = self._tokens[idx]
        sample = {
            'source': [self._dictionary.bos_token] + tokens,
            'target': tokens + [self._dictionary.eos_token]
        }
        if self.transform:
            sample = self.transform(sample)
        return sample
        
    def __len__(self):
        return len(self._tokens)

In [ ]:
def seq_collate_fn(data):
    
    def padding(seqs, seq_lens, dtype=torch.LongTensor):
        batch_size = len(seqs)
        max_seq_len = max(seq_lens)
        source_padded = torch.zeros(batch_size, max_seq_len).type(dtype)
        target_padded = torch.zeros(batch_size, max_seq_len).type(dtype)
        for i, seq in enumerate(seqs):
            end = len(seq['source'])
            source_padded[i,:end] = seq['source'] 
            target_padded[i,:end] = seq['target']
        
        return source_padded, target_padded
        
    data.sort(key=lambda d: len(d['source']), reverse=True)
    seq_lens = [len(d['source']) for d in data]
    source_padded, target_padded = padding(data, seq_lens)
    samples = {
        'sources': source_padded,
        'targets': target_padded,
        'seq_lens': seq_lens,
    }
    
    return samples

In [ ]:
class SentenceDataModule(pl.LightningDataModule):
    def __init__(self, path: str, batch_size: int, num_workers: int):
        super().__init__()
        self.data_path = path
        self.batch_size = batch_size 
        self.num_workers = num_workers

        self.train_path = os.path.join(path, 'train.txt')
        self.val_path = os.path.join(path, 'valid.txt')
        self.test_path = os.path.join(path, 'test.txt')
        self.all_paths = [self.train_path, self.val_path, self.test_path]
        self.transforms = ToTensor()
        self.vocab = self._create_vocab(self.all_paths)
        self.dims = len(self.vocab)
        
    
    def prepare_data(self):
        pass

    def setup(self, stage = None):
        print(f"In Setup, with stage: {stage}")
        #if stage == 'fit' or stage is None:
        self.train_dataset = SentenceDataset(self.train_path, self.vocab, 
                                                 transform=self.transforms)
        self.val_dataset = SentenceDataset(self.val_path, self.vocab,
                                               transform=self.transforms)
        #if stage == 'test' or stage is None:
        self.test_dataset = SentenceDataset(self.test_path, self.vocab,
                                                transform=self.transforms)


    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, 
                          num_workers=self.num_workers, collate_fn=seq_collate_fn)
        
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, 
                          num_workers=self.num_workers, collate_fn=seq_collate_fn)
        
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, 
                          num_workers=selfnum_workers, collate_fn=seq_collate_fn)

    def _create_vocab(self, paths):
        vocab = Dictionary()
        for path in paths:
            # Add words to the dictionary
            with open(path, 'r', encoding="utf8") as f: 
                for line in f:
                    if len(line.strip()) == 0:
                        continue
                    words = [word.lower() for word in line.strip().split()]
                    for word in words:
                        vocab.add_word(word)
        return vocab
        

In [ ]:
dm = SentenceDataModule('drive/My Drive/wiki-2', BATCH_SIZE, NUM_WORKERS)
lm = RNNLanguageModel(dm.size(), 256, dropout_prob=0.2, num_layers=4)
trainer = pl.Trainer(gpus=AVAIL_GPUS, max_epochs=5, progress_bar_refresh_rate=2, logger=wandb_logger)

In [ ]:
trainer.fit(lm, dm)

In [ ]:
wandb.finish()